ElGamal
===============

## Diffie-Hellman key distribution scheme

IN 1975, Diffie and Hellman introduced the concept of public key cryptography.

Suppose that $A$ and $B$ want to share a secret $K_{AB}$ , where A has a secret $x_A$ and $B$ has a secret $x_B$ . Let $p$ be a large prime and $\alpha$ be a **primitive element** mod $p$, both known. computes $y \equiv \alpha^{x_a} \mod p$ , and sends $y_A$ . Similarly, $B$ computes $y_B \equiv \alpha^{x_B} \mod p$ and sends $y_B$ . Then the secret $K_{AB}$ is computed as:

\begin{align}
K_{AB} &\equiv \alpha^{x_ax_B} \mod p\\
&\equiv y_A^{x_B} \mod p\\
&\equiv y_B^{x_A} \mod p
\end{align}

In any of the cryptographic systems based on discrete logarithms, p must be chosen such that $p - 1$ has at least one large prime factor. If $p - 1$ has only small prime factors, then computing discrete logarithms is easy

In [1]:
from klefki.numbers.primes import generate_prime
from klefki.types.algebra.meta import field
from klefki.types.algebra.utils import randfield
from math import gcd

In [2]:
p = generate_prime(32)
F = field(p)
mF = field(p-1)
G = F(generate_prime(32))

In [3]:
xa, xb = randfield(F).value, randfield(F).value
ya, yb = G ** xa, G ** xb
G ** (xa * xb) == ya ** xb == yb ** xa

True

## Classic ElGamal

Let m be a document to be signed, where $0 \le m \ge p - 1$. The public file still consists of the public key $y \equiv x \mod p$ for each user.

To sign a document, a user $A$ should be able to use the secret key $x_A$ to find a signature for $m$ in such a way that all users can verify the authenticity of the signature by using the public key $y_A$ (together with $\alpha$ and $p$), and no one can forge a signature without knowing the secret $x_A$ .

The signature for $m$ is the pair $(r, s), 0 \le r, s < p - 1$, chosen such that the equation

$$
\alpha^m \equiv y^r r^s \mod p
$$

is satisfied.

In [4]:
m = randfield(F).value
alpha = G
x = xa
y = ya

* The Signing Procedure

The signing procedure consists of the following three steps.

1) Choose a random number $k$, uniformly between $0$ and $p - 1$, such that $gcd(k, p - 1) = 1$.

2) Compute

$$
r \equiv \alpha^k \mod p\\
\alpha^m \equiv \alpha^{xr}\alpha^{ks} \mod p
$$

which can be solved for $s$ by using

$$
m \equiv xr + ks \mod (p-1)
$$

Above Equation as a solution for $s$ if $k$ is chosen such that $gcd(k, p - 1) = 1$.

In [5]:
k = generate_prime(32)
assert gcd(k, p-1) == 1

In [6]:
from klefki.numbers import invmod
r = (alpha ** k).value
s = ((mF(m) - mF(r) * mF(x)) * ~mF(k)).value


In [7]:
assert pow(alpha, m) ==  pow(alpha, (x * r + k * s))

* The Verification Procedure

Given $m, r$, an $s$, it is easy to verify the authenticity of the signature by computing both sides of $\alpha^m \equiv y^r r^s \mod p$ and checking that they are equal.

In [8]:
alpha ** m == y ** r * F(r) ** s

True

## ElGamal over Cyclic Group

In [10]:
from klefki.types.algebra.concrete import EllipticCurveCyclicSubgroupSecp256k1 as Curve
from klefki.types.algebra.meta import field

* Key generation

The first party, Alice, generates a key pair as follows:

* Generate an efficient description of a cyclic group $G$, of order $q$  with generator $g$. Let $e$ represent the unit element of $G$.

* Choose an integer $x\leftarrow Z_q$

* Compute $h:=G^x$

* he public key consists of the values $( G , q , g , h )$. Alice publishes this public key and retains $x$ as her private key, which must be kept secret.

In [14]:
G = Curve.G
F = field(G.N)
x = randfield(F).value
h = G ** x


* Encryption

A second party, Bob, encrypts a message $M$ to Alice under her public key $( G , q , g , h )$  as follows: 

* Map the message $M$ to an element $m$ of $G$ using a reversible mapping function.

* Choose an integer $y$ randomly from $\{ 1 , … , q − 1 \}$ 

* Compute $s := h^y$. This is called the shared secret.

* Compute $c_1 := g^y$

* Compute $c_2 := m \cdot s$

Bob sends the ciphertext $( c_1 , c_2 )$ to Alice.

In [20]:
y = randfield(F).value
M = randfield(F)
m = Curve.lift_x(M)
s = h ** y
c1 = G ** y
c1 = s * m

* Decryption

Alice decrypts a ciphertext $c_1, c_2$ with her private key $x $ as follows: 

* Compute $s := c_1^x$.

* Compute $s^{-1}$

* Compute $m:=c_2\cdot s^{-1}$


## Ref:
    
   * T. ElGamal. A Public Key Cryptosystem and a Signature Scheme Based on Discrete Logarithms. IEEE Trans. Info. Theory, IT 31:469–472, 1985.
    
   * ElGamal encryption https://en.wikipedia.org/wiki/ElGamal_encryption
    
   * [1] Primitive element (finite field) https://en.wikipedia.org/wiki/Primitive_element_(finite_field)